In [49]:
## Packages 

import csv
import pandas as pd
import numpy as np
import os
import random

# Progressbar
from tqdm.keras import TqdmCallback

from tqdm import tqdm # progess bar
import pickle

# For graphs
import networkx as nx 
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, HinSAGE, link_classification
from stellargraph import globalvar

from stellargraph.layer import MeanAggregator

# For DL
from tensorflow import keras 
######
import tensorflow as tf

config = tf.ConfigProto(intra_op_parallelism_threads=8, inter_op_parallelism_threads=2, allow_soft_placement=True, device_count = {'CPU': 8})

session = tf.Session(config=config)

os.environ["OMP_NUM_THREADS"] = "8"

os.environ["KMP_BLOCKTIME"] = "30"

os.environ["KMP_SETTINGS"] = "1"

os.environ["KMP_AFFINITY"]= "granularity=fine,verbose,compact,1,0"

#####




# For processing node texts
#import spacy
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction import text as fe

# Dimensionality reduction
from sklearn.decomposition import NMF, LatentDirichletAllocation

# Word embeddings
#import gensim 
#from gensim.models import Word2Vec

# For stemming
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize


import random
import numpy as np
import pandas as pd
import csv
from tqdm import tqdm
import os
import networkx as nx
import pdb
import pickle
from collections import Counter
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.feature_extraction import text as fe
from sklearn.decomposition import NMF, LatentDirichletAllocation
import matplotlib.pyplot as plt
import datetime
import time
import keras
import lightgbm
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
import nltk



In [50]:
path = r"pickles/word_matrix300id.PICKLE"
with open(path, 'rb') as f:
    word_matrix = pickle.load(f)
    word_matrix = np.asarray(word_matrix)
f.close()

# Storing in dataframe
n1, n2 = word_matrix.shape
# Creating feature names
feature_names = ["w_{}".format(ii) for ii in range(n2)]
ids = sorted(range(n1), key=str)
node_data = pd.DataFrame(data=word_matrix, index=ids, columns=feature_names)
print(n1,n2)

33226 500


In [51]:
with open(r"training.txt", "r") as f:
    reader = csv.reader(f)
    training  = list(reader)
# in order of training examples
training = [element[0].split(" ") for element in training]
training = pd.DataFrame(training, columns=['Node1', 'Node2', 'Link'])
print("Training examples shape: {}".format(training.shape))

with open(r"testing.txt", "r") as f:
    reader = csv.reader(f)
    testing  = list(reader)
# in order of testing examples
testing = [element[0].split(" ") for element in testing]
testing = pd.DataFrame(testing, columns=['Node1', 'Node2'])
print("Testing examples shape: {}".format(testing.shape))

Training examples shape: (453797, 3)
Testing examples shape: (113450, 2)


In [52]:
feature_vector_path = r"pickles/perf_val_data_final.PICKLE"
if os.path.exists(feature_vector_path):
    with open(feature_vector_path, 'rb') as f:
        perf_val_data = pickle.load(f)
    f.close()

feature_vector_path = r"pickles/perf_test_data_final.PICKLE"
if os.path.exists(feature_vector_path):
    with open(feature_vector_path, 'rb') as f:
        perf_test_data = pickle.load(f)
    f.close()

In [84]:
def average(index, dataset):
    node1 = dataset['Node1'][index]
    node2 = dataset['Node2'][index]
    index1 = ids.index(int(node1))
    index2 = ids.index(int(node2))
    
    sim = (node_data.loc[index1]+node_data.loc[index2])/2 # Change binary operator to whatever here ....
    return sim.to_numpy()

feature_names = ["doc2vec_average{}".format(ii) for ii in range(500)]

perf_val_data['doc2vec_average']=list(map(lambda i:average(i, training), 
                                              tqdm(perf_val_data['original_index'], position=0, leave=True)))
perf_test_data['doc2vec_average']=list(map(lambda i:average(i, testing), 
                                               tqdm(perf_test_data['original_index'], position=0, leave=True)))




100%|██████████| 113450/113450 [01:54<00:00, 987.29it/s] 


In [85]:
with open(feature_vector_path, '+wb') as f:
    pickle.dump(perf_val_data, f)
f.close()

with open(feature_vector_path, '+wb') as f:
    pickle.dump(perf_test_data, f)
f.close()

In [86]:
cols = ['doc2vec_average']
X = pd.DataFrame(perf_val_data[cols].doc2vec_average.tolist(), columns=feature_names)
y = perf_val_data['Link']
y = list(map(lambda i: int(i), y))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

X_train

,doc2vec_average0,doc2vec_average1,doc2vec_average2,doc2vec_average3,doc2vec_average4,doc2vec_average5,doc2vec_average6,doc2vec_average7,doc2vec_average8,doc2vec_average9,...,doc2vec_average490,doc2vec_average491,doc2vec_average492,doc2vec_average493,doc2vec_average494,doc2vec_average495,doc2vec_average496,doc2vec_average497,doc2vec_average498,doc2vec_average499
21155,-0.020063,0.052347,-0.058800,0.260004,0.823371,0.124695,-0.051623,-0.095660,-0.171536,0.119674,...,0.035858,0.071236,-0.007832,-0.124876,-0.031379,0.249358,-0.013508,0.014364,0.003491,-0.007654
2879,0.394569,-0.389487,-0.055986,-0.213530,-0.097315,-0.491774,0.542253,0.563140,1.171107,0.003436,...,-0.418845,0.298539,-0.031765,-0.624266,0.024696,0.103616,0.077364,-0.793604,0.443700,-0.069849
5038,0.042739,-0.289233,0.020959,-0.218276,-0.099425,0.010675,-0.050894,0.031364,-0.008077,-0.049247,...,0.048522,0.010676,-0.000310,-0.125034,0.009019,0.034088,0.004515,-0.029699,-0.055352,0.140198
4736,-0.079235,-0.219151,-0.019431,0.018875,-0.077348,0.041631,0.081416,0.013799,-0.019787,0.060417,...,0.015948,0.007363,-0.129307,-0.250856,0.016768,-0.016657,0.441415,0.273992,0.047529,0.049225
8588,1.118262,0.385503,-0.742771,0.283396,-0.365477,-0.145103,-0.024366,0.142328,0.024896,-0.010650,...,-0.100515,0.228956,0.418196,0.209789,0.011820,-0.102237,-0.636528,-0.132150,0.134350,0.037607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4574,0.037439,0.054001,0.021581,0.271279,-0.008193,0.100828,0.067756,-0.026939,-0.035822,0.041048,...,0.130872,-0.002113,-0.443449,0.006641,0.015395,-0.005536,-0.000640,0.024159,0.086958,-0.018980
20589,0.259838,-0.213578,-0.374419,0.209607,0.459305,0.387589,0.333880,-0.394993,0.020934,-0.439540,...,-1.407241,0.304945,-0.011838,-1.239350,0.038491,-1.020851,0.016780,-0.655348,0.128691,-0.473293
19417,-0.193987,1.043525,-0.000199,0.624679,0.441421,-0.170685,0.159183,0.122273,-0.040909,0.427975,...,0.055863,-0.057339,-0.004505,0.346912,0.384794,-0.180021,0.085751,-0.043945,0.739188,-0.221828
16049,-0.013320,0.004675,-0.006543,0.152056,0.067065,0.001512,0.069608,0.074682,-0.441345,0.001332,...,0.069179,-0.044413,0.002426,0.016385,-0.337567,0.547179,-0.004212,-0.001307,0.061947,-0.103081


In [88]:
lgbm = lightgbm.LGBMClassifier(verbose=1)
model_lgbm = lgbm.fit(X_train, y_train)
predictions = model_lgbm.predict(X_test)
print(f1_score(y_test, predictions))

0.7486445456517025
